# 6. 케라스(Keras) 훑어보기

## 1) 전처리(Preprocessing)
- **Tokenizer()** : 토큰화와 정수 인코딩(단어에 대한 인덱싱)을 위해 사용
- **pad_sequence()** : 보통 숫자 0을 넣어서 길이가 다른 샘플들의 길이를 맞춰주는 작업. 정해준 길이보다 길이가 긴 샘플은 일부 자름

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
t  = Tokenizer()
fit_text = "The earth is an awesome place live"
t.fit_on_texts([fit_text])

test_text = "The earth is an great place live"
sequences = t.texts_to_sequences([test_text])[0]

print("sequences : ",sequences) # great는 단어 집합(vocabulary)에 없으므로 출력되지 않는다.
print("word_index : ",t.word_index) # 단어 집합(vocabulary) 출력

sequences :  [1, 2, 3, 4, 6, 7]
word_index :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], maxlen=3, padding='pre')
# 전처리가 끝나서 각 단어에 대한 정수 인코딩이 끝났다고 가정하고, 3개의 데이터를 입력으로 합니다.

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]])

## 2) 워드 임베딩(Word Embedding)
> 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것

- 희소벡터(sparse vector) : 대부분의 값이 0인 벡터
- 밀집벡터(dense vecor) : 대부분의 값이 0이 아닌 실수이며 상대적으로 저차원인 벡터

|-|원-핫 벡터|임베딩 벡터|
|---|---|---|
|차원|고차원(단어 집합의 크기)|저차원|
|다른 표현|희소 벡터의 일종|밀집 벡터의 일종|
|표현 방법|수동|훈련 데이터로부터 학습함|
|값의 타입|1과 0|실수|

- **워드 임베딩(word embedding)** :  밀집 벡터로 만드는 작업
- **Embedding(number of samples, input_length)** : 
    - 단어를 밀집 벡터로 만드는 역할.
    - 정수 인코딩이 된 단어들을 입력을 받아서 임베딩을 수행
    - 워드 임베딩 작업을 수행하고 (number of samples, input_length, embedding word dimentionality)인 3D 텐서를 리턴

## 3) 모델링(Modeling)
- **Sequential()** : 모델링 시작
- **model.add()** : 층 추가
    - Embedding() : 임베딩 층 추가
    - Dense() : fully-connected 층 추가
    

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Dense(1, input_dim=3, activation='relu'))

<img src=neural_network1_final.png width=300>

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 출력층

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<img src=neural_network2_final.png width=300>

In [5]:
# 위의 코드의 연장선상에 있는 코드임.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 40        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


## 4) 컴파일(Compile)과 훈련(Training)
- **compile()** : 모델의 옵션 설정(오차함수, 최적화 방법, 메트릭 함수)
- **fit()** : 모델을 학습

In [ ]:
# 이 코드는 뒤의 텍스트 분류 챕터의 스팸 메일 분류하기 실습 코드를 갖고온 것임.
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
max_features = 10000

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32)) #RNN에 대한 설명은 뒤의 챕터에서 합니다.
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

|문제유형|loss function|activation function|참고 설명|
|---|---|---|---|
|회귀문제|mean_squred_error(평균 제곱 오차)|-|-|
|다중클래스 분류|categorical_crossentropy(범주형 교차 엔트로피)|소프트맥스|Ch.10 로이터뉴스 분류하기 실습|
|다중클래스 분류|sparse_categorical_crossentropy|소프트맥스|정수 인코딩 된 상태에서 수행 가능|
|이진분류|binary_crossentropy(이항 교차 엔트로피)|시그모이드|Ch.10 스팸 메일 분류하기|

In [ ]:
model.fit(X_train, y_train, epochs=10, 
          batch_size=32, verbose=0, validation_data(X_val, y_val))

In [ ]:
# 훈련 데이터의 20%를 검증 데이터로 사용.
model.fit(X_train, y_train, epochs=10, 
          batch_size=32, verbose=0, validation_split=0.2)

## 5) 평가(Evaluation)와 예측(Prediction)
- **evaluate()** : 모델의 정확도 평가
- **predict()** : 임의의 입력에 대한 모델의 예측값(출력값)

In [ ]:
# 위의 fit() 코드의 연장선상인 코드
model.evaluate(X_test, y_test, batch_size=32)

# 위의 fit() 코드의 연장선상인 코드
model.predict(X_input, batch_size=32)

## 6) 모델의 저장(Save)과 로드(Load)
- **save()** : 인공신경망 모델을 hdf5 파일에 저장
- **load_model()** : 저장해둔 모델 불러오기

In [ ]:
model.save("model_name.h5")

from tensorflow.keras.models import load_model
model = load_model("model_name.h5")